In [1]:
import pandas as pd
from flask import Flask, request, jsonify
import threading
from flask_cors import CORS

In [2]:
df2019 = pd.read_csv("conteos_mensuales2019.csv")
df2019["estado"] = df2019["estado"].str.upper().str.replace(" ", "")
# Agrupar por fecha y entidad, y sumar los valores
df2019 = df2019.groupby(["year_month", "estado"], as_index=False).sum()
# Separar la columna de fecha en año y mes
df2019[["Año", "Mes"]] = df2019["year_month"].str.split("-", expand=True)

# Eliminar la columna original de fecha
df2019 = df2019.drop(columns=["year_month"])

In [3]:
df2020 = pd.read_csv("conteos_mensuales2020.csv")
df2020["estado"] = df2020["estado"].str.upper().str.replace(" ", "")
# Agrupar por fecha y entidad, y sumar los valores
df2020 = df2020.groupby(["year_month", "estado"], as_index=False).sum()
# Separar la columna de fecha en año y mes
df2020[["Año", "Mes"]] = df2020["year_month"].str.split("-", expand=True)

# Eliminar la columna original de fecha
df2020 = df2020.drop(columns=["year_month"])

In [4]:
df2021 = pd.read_csv("conteos_mensuales_2021.csv")
df2021["estado"] = df2021["estado"].str.upper().str.replace(" ", "")
# Agrupar por fecha y entidad, y sumar los valores
df2021 = df2021.groupby(["year_month", "estado"], as_index=False).sum()
# Separar la columna de fecha en año y mes
df2021[["Año", "Mes"]] = df2021["year_month"].str.split("-", expand=True)

# Eliminar la columna original de fecha
df2021 = df2021.drop(columns=["year_month"])

In [5]:
df2022 = pd.read_csv("conteos_mensuales_2022.csv")
df2022["estado"] = df2022["estado"].str.upper().str.replace(" ", "")
# Agrupar por fecha y entidad, y sumar los valores
df2022 = df2022.groupby(["year_month", "estado"], as_index=False).sum()
# Separar la columna de fecha en año y mes
df2022[["Año", "Mes"]] = df2022["year_month"].str.split("-", expand=True)

# Eliminar la columna original de fecha
df2022 = df2022.drop(columns=["year_month"])

In [6]:
app = Flask(__name__)
CORS(app)

In [7]:
datasets = {"2019": df2019, "2020": df2020, "2021": df2021, "2022": df2022}

In [8]:
@app.route("/get_data_by_year", methods=["POST"])
def get_data_by_year():
    request_data = request.get_json()
    estado = request_data.get("estado").upper().replace(" ", "")
    año = request_data.get("año")

    if año not in datasets:
        return jsonify({"error": "Año no válido"}), 400

    df = datasets[año]
    result = df[df["estado"] == estado]

    if result.empty:
        return (
            jsonify(
                {"error": "No se encontraron datos para los parámetros proporcionados"}
            ),
            404,
        )

    result = result.to_dict(orient="records")
    return jsonify(result), 200

In [9]:
@app.route("/get_data", methods=["POST"])
def get_data():
    request_data = request.get_json()
    estado = request_data.get("estado").upper().replace(" ", "")
    mes = request_data.get("mes")
    año = request_data.get("año")

    if año not in datasets:
        return jsonify({"error": "Año no válido"}), 400

    df = datasets[año]
    result = df[(df["estado"] == estado) & (df["Mes"] == mes)]

    if result.empty:
        return (
            jsonify(
                {"error": "No se encontraron datos para los parámetros proporcionados"}
            ),
            404,
        )

    result = result.to_dict(orient="records")
    return jsonify(result), 200


In [ ]:
def run_app():
    app.run(debug=False, use_reloader=False)


# Ejecutar el servidor Flask en un hilo separado
threading.Thread(target=run_app).start()

 * Serving Flask app '__main__'
 * Debug mode: off


 * Running on http://127.0.0.1:5000
Press CTRL+C to quit


In [11]:
import requests
import json

url = "http://127.0.0.1:5000/get_data_by_year"
headers = {"Content-Type": "application/json"}
data = {"estado": "Ciudad de México", "año": "2019"}

response = requests.post(url, headers=headers, data=json.dumps(data))
print(response.json())

127.0.0.1 - - [13/Jun/2024 00:48:18] "POST /get_data_by_year HTTP/1.1" 200 -


[{'Año': '2019', 'Mes': '01', 'conteo': 341, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '02', 'conteo': 397, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '03', 'conteo': 544, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '04', 'conteo': 400, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '05', 'conteo': 405, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '06', 'conteo': 501, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '07', 'conteo': 618, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '08', 'conteo': 892, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '09', 'conteo': 718, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '10', 'conteo': 771, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '11', 'conteo': 579, 'estado': 'CIUDADDEMÉXICO'}, {'Año': '2019', 'Mes': '12', 'conteo': 761, 'estado': 'CIUDADDEMÉXICO'}]
